# Base íris, classificação multiclasse com kfold

Vamos fazer a mesma coisa que fizemos na implementação da rede neural da Íris, porém 
utilizando da validação cruzada

**Obs.: esse jupyter notebook não consegue ser rodado por conta do excesso de memória gasto, neste caso, rode o script em python**

In [1]:
import pandas as pd

base       = pd.read_csv("iris.csv")
previsores = base.iloc[:,0:4].values
classe     = base.iloc[:,4].values

In [2]:
from sklearn.preprocessing import LabelEncoder 
from keras.utils import np_utils

# Transformar os dados em String para tipo numérico
label_enconder = LabelEncoder()
classe = label_enconder.fit_transform(classe)

# Transformar o array de uma dimensão em três dimensões, para se ajustar 
# com o modelo de saida que tem três possibilidades
classe_dummy = np_utils.to_categorical(classe)

2022-08-24 20:36:38.702894: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-24 20:36:38.702909: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Novamente, vamos colocar a lógica de criar a rede neural em uma função

In [3]:
from keras.models import Sequential
from keras.layers import Dense 

def create_neural_networks():
    classificador = Sequential()

    camada_escondida = Dense(units = 4,
                            activation = 'relu',
                            input_dim = 4)

    camada_escondida2 = Dense(units = 4,
                            activation = 'relu')

    """
    Diferente da classificação binária, neste caso temos 
    três neurônios na camada de saída, pois temos três possíveis
    resultados. Além disso, a nossa função de ativação será a softmax, pois
    queremos transformar um vetor de dados (neste caso as informações processadas
    das petalas), em um valor número probabilístico.
    """
    camada_de_saida = Dense(units = 3, 
                            activation = 'softmax')

    classificador.add(camada_escondida)
    classificador.add(camada_escondida2)
    classificador.add(camada_de_saida)

    classificador.compile(
        optimizer="adam", 
        loss="categorical_crossentropy", 
        metrics=["categorical_accuracy"])

    return classificador

Por fim, vamos criar o classificador e fazer a validação cruzada

In [4]:
from keras.wrappers.scikit_learn import KerasClassifier

classificador = KerasClassifier(build_fn=create_neural_networks,
                                epochs=1000,
                                batch_size=10)

/tmp/ipykernel_232677/433920566.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classificador = KerasClassifier(build_fn=create_neural_networks,


In [8]:
from sklearn.model_selection import cross_val_score

resultados = cross_val_score(estimator=classificador,
                            X=previsores,
                            y=classe,
                            cv=10,
                            scoring='accuracy')

/home/guaruda/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/guaruda/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/guaruda/.local/lib/python3.10/site-packages/keras/wrappers/scikit_learn.py", line 236, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/home/guaruda/.local/lib/python3.10/site-packages/keras/wrappers/scikit_learn.py", line 157, in fit
    if (losses.is_ca

Aqui vamos imprimir o resultado da média e do desvio

In [ ]:
media = resultados.mean()
desvio = resultados.std()